In [ ]:
import cudf
import dask_cudf

#PROTO-LOGIC BEHIND THE BOLLINGER BANDS PANDAS VERSION.




# Initialize positions
df['positions'] = 0

# Generate signals
df['long_entry'] = (df['price'] < df['lower_band'])
df['long_exit'] = (df['price'] > df['middle_band'])
df['short_entry'] = (df['price'] > df['upper_band'])
df['short_exit'] = (df['price'] < df['middle_band'])

# Create position column
df.loc[df['long_entry'], 'positions'] = 1
df.loc[df['long_exit'], 'positions'] = 0
df.loc[df['short_entry'], 'positions'] = -1
df.loc[df['short_exit'], 'positions'] = 0

# Forward fill positions
df['positions'] = df['positions'].fillna(method='ffill').fillna(0)

# Record entry price
df['entry_price'] = df['price'].where(df['positions'].diff() != 0).fillna(method='ffill')

# Set stop loss percentage
stop_loss_percentage = 0.05
# Apply stop loss using cuDF
df.loc[(df['positions'] == 1) & (df['price'] < df['entry_price'] * (1 - stop_loss_percentage)), 'positions'] = 0
df.loc[(df['positions'] == -1) & (df['price'] > df['entry_price'] * (1 + stop_loss_percentage)), 'positions'] = 0

# Recalculate positions after stop loss
df['positions'] = df['positions'].fillna(method='ffill').fillna(0)

# Calculate returns
df['market_returns'] = df['price'].pct_change()
df['strategy_returns'] = df['market_returns'] * df['positions'].shift(1)